In [ ]:
import numpy as np
import pandas as pd
import os

In [ ]:
#pip list
import sklearn
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import f1_score,accuracy_score
import transformers
import torch
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertModel, BertConfig

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install -qq simpletransformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.5/315.5 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 6.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 41.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.5/258.5 kB 36.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 21.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 73.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 13.

In [ ]:
dataset=pd.read_csv("/content/drive/MyDrive/Capstone/Dataset/Preprocessed/tamil_dataset_2_train.csv")
X=list(dataset['text'])
y=list(dataset['label'])
X_train, X_test, y_train, y_test = train_test_split(X, y , random_state=42,test_size=0.3)
train_df = pd.DataFrame({"text": X_train, "label": y_train})
test_df = pd.DataFrame({"text": X_test, "label": y_test})

In [ ]:
dataset1=pd.read_csv("/content/drive/MyDrive/Capstone/Dataset/Preprocessed/tamil_dataset_1_train.csv")
dataset1_filtered = dataset1[dataset1['label'] != 2]
X_train=list(dataset1_filtered['text'])
y_train=list(dataset1_filtered['label'])
train_df = pd.DataFrame({"text": X_train, "label": y_train})
dataset2=pd.read_csv("/content/drive/MyDrive/Capstone/Dataset/Preprocessed/tamil_dataset_1_test.csv")
dataset2_filtered = dataset2[dataset2['label'] != 2]
X_test=list(dataset2_filtered['text'])
y_test=list(dataset2_filtered['label'])
test_df = pd.DataFrame({"text": X_test, "label": y_test})

In [ ]:
train_df.shape

(33685, 2)

In [ ]:
test_df.shape

(4216, 2)

In [ ]:
import torch
torch.cuda.device_count()

1

In [ ]:
device = torch.cuda.get_device_properties(0)  # Assuming you have at least one GPU
print(device)

_CudaDeviceProperties(name='Tesla T4', major=7, minor=5, total_memory=15102MB, multi_processor_count=40)


In [ ]:
colab_sample_data_path = '/content/sample_data'
os.chdir(colab_sample_data_path)

In [ ]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs

In [ ]:
#clear kaggle/working
import shutil

# Clear the contents of the working directory
shutil.rmtree('/content/sample_data/cache_dir', ignore_errors=True)
shutil.rmtree('/content/sample_data/outputs', ignore_errors=True)
shutil.rmtree('/content/sample_data/runs', ignore_errors=True)

In [ ]:
import pickle
tbert="/content/drive/MyDrive/Capstone/Models/Tamil/tbert_tamil.sav"
tbert_model_tamil=pickle.load(open(tbert,'rb'))

# Tamil_BERT

In [ ]:
train_args = {
    "reprocess_input_data": True,
    "fp16": False,
    "num_train_epochs": 20,
    "early_stopping_metric": "eval_loss",
    "early_stopping_metric_minimize": True,
    "early_stopping_patience": 2,
    'overwrite_output_dir': True,
    'optimizer': 'AdamW',
    'train_batch_size': 64,
    "learning_rate":2e-5,
    "gradient_accumulation_steps":2,
    "use_early_stopping": True,
}

tbert_model_tamil = ClassificationModel(
    "bert",
    "l3cube-pune/tamil-bert",
    num_labels=2,
    args=train_args,
    use_cuda=True
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/660 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/951M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at l3cube-pune/tamil-bert and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/450 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/3.16M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/6.41M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [ ]:
tbert_model_tamil.train_model(train_df)

  0%|          | 0/58 [00:00<?, ?it/s]

Epoch:   0%|          | 0/20 [00:00<?, ?it/s]

Running Epoch 1 of 20:   0%|          | 0/461 [00:00<?, ?it/s]

Running Epoch 2 of 20:   0%|          | 0/461 [00:00<?, ?it/s]

Running Epoch 3 of 20:   0%|          | 0/461 [00:00<?, ?it/s]

...

Running Epoch 17 of 20:   0%|          | 0/461 [00:00<?, ?it/s]

Running Epoch 18 of 20:   0%|          | 0/461 [00:00<?, ?it/s]

Running Epoch 19 of 20:   0%|          | 0/461 [00:00<?, ?it/s]

Running Epoch 20 of 20:   0%|          | 0/461 [00:00<?, ?it/s]

(4600, 0.21169700572768266)

In [ ]:
# Evaluate the model
result, model_outputs, wrong_predictions = tbert_model_tamil.eval_model(test_df)

/usr/local/lib/python3.10/dist-packages/simpletransformers/classification/classification_model.py:1453: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


  0%|          | 0/8 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/43 [00:00<?, ?it/s]

In [ ]:
print(result) #result of the model

{'mcc': 0.8448231136071761, 'accuracy': 0.943785578747628, 'f1_score': 0.8810837932764677, 'tp': 878, 'tn': 3101, 'fp': 92, 'fn': 145, 'auroc': 0.9596404524927604, 'auprc': 0.9086341812963605, 'eval_loss': 0.23967351088690203}


In [ ]:
predictions, raw_outputs = tbert_model_tamil.predict(list(test_df['text'].values))

  0%|          | 0/8 [00:00<?, ?it/s]

  0%|          | 0/43 [00:00<?, ?it/s]

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(predictions,test_df['label'].values,zero_division=True))

              precision    recall  f1-score   support

           0       0.97      0.96      0.96      3246
           1       0.86      0.91      0.88       970

    accuracy                           0.94      4216
   macro avg       0.91      0.93      0.92      4216
weighted avg       0.95      0.94      0.94      4216



In [ ]:
import pickle
# save the model to disk
filename = 'tbert_tamil.sav'
pickle.dump(tbert_model_tamil, open(filename, 'wb'))

In [ ]:
# load the model from disk
loaded_model = pickle.load(open(filename, 'rb'))

In [ ]:
sample_text="Ethu padama ah ila serial ah"
loaded_model.predict([sample_text])[0]

0it [00:00, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

array([1])

In [ ]:
sample_text="Ethu padame illa, pakka serial. mokka padam"
loaded_model.predict([sample_text])[0]

0it [00:00, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

array([1])

In [ ]:
sample_text="அந்த உணவகத்தில் உணவும் நன்றாக இருந்தது"
loaded_model.predict([sample_text])[0]

0it [00:00, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

array([0])

In [ ]:
accuracy = accuracy_score(test_df['label'], predictions)
print("Accuracy:", accuracy)

Accuracy: 0.943785578747628


In [ ]:
f1 = f1_score(test_df['label'], predictions,average="weighted")
print("F1 score:", f1)

F1 score: 0.9432694745963669


In [ ]:
import shutil

local_model_path = '/content/sample_data/tbert_tamil.sav'
destination_path = '/content/drive/MyDrive/Capstone/Models/Tamil/tbert_tamil.sav'

# Copy the file to the destination directory
shutil.copy(local_model_path, destination_path)


'/content/drive/MyDrive/Capstone/Models/Tamil/tbert_tamil.sav'